In [ ]:
from tensorflow.keras import Sequential, layers, optimizers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.distribute import MirroredStrategy, ReductionToOneDevice, HierarchicalCopyAllReduce
from tensorflow import math
import tensorflow_addons as tfa

In [ ]:
data_dir = "/home/jovyan/data-vol-1/face_chip_clusters"
batch_size=64
input_shape = (224, 224, 3)
img_shape = input_shape[:2]
print(img_shape)

In [ ]:
train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_shape,
    batch_size=batch_size)

eval_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    shuffle=False,
    image_size=img_shape,
    batch_size=batch_size)

In [ ]:
strategy = MirroredStrategy(cross_device_ops=HierarchicalCopyAllReduce())
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

with strategy.scope():
    model = Sequential(
        [
            layers.Conv2D(64, 3, padding='same', activation='relu', input_shape=input_shape),
            layers.MaxPooling2D(pool_size=2),
            layers.Dropout(0.3),
            layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'),
            layers.MaxPooling2D(pool_size=2),
            layers.Dropout(0.3),
            layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'),
            layers.MaxPooling2D(pool_size=2),
            layers.Dropout(0.3),
            layers.Flatten(),
            layers.Dense(256, activation=None),
            layers.Lambda(
                lambda x: math.l2_normalize(x, axis=1)
            ),
        ]
    )
    model.compile(
        optimizer=optimizers.Adam(0.001), loss=tfa.losses.TripletSemiHardLoss()
    )
model.summary()

In [ ]:
history = model.fit(
    train_ds,
    epochs=15)

In [ ]:
results = model.predict(eval_ds)
model.save("/home/jovyan/data-vol-1/brandon_face_model")

In [ ]:
import numpy as np
import io
import tensorflow_datasets as tfds

In [ ]:
np.savetxt("face_model/vecs.tsv", results, delimiter='\t')
out_m = io.open('face_model/meta.tsv', 'w', encoding='utf-8')
for img, labels in tfds.as_numpy(eval_ds):
    [out_m.write(str(x) + "\n") for x in labels]
out_m.close()